In [11]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [12]:
vvr_data_world_file = "/Users/rajee/gt/RAM-Project/RAM_project/Resources/vegetarian_restaurants_US_datafiniti_vegetarian_restaurants_US_datafiniti.csv"
vvr_df = pd.read_csv(vvr_data_world_file)
vvr_indexed_df = vvr_df.reset_index(drop=False, inplace=False, col_fill='')
vvr_indexed_df.head()

,index,address,categories,city,claimed,country,cuisines,dateOpened,dateUpdated,descriptions,...,menuURL,name,paymentTypes,phones,postalCode,priceRange,province,sic,twitter,websites
0,0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,NaN,US,NaN,NaN,2014-02-01T04:41:06Z,"Tags: American (New), American, and Trendy Sta...",...,NaN,Village Inn Restaurant and Bakery,NaN,7277964988,33759,NaN,FL,8423,NaN,NaN
1,1,605 W Main St,Caf and Sandwich Place,Louisville,NaN,US,"Vegetarian, Delicatessen",NaN,2016-07-23T22:37:24Z,"Atlantic No. 5, Louisville: See 4 unbiased rev...",...,NaN,Atlantic No. 5,Yes,5028833398,40202,USD 25.00-40.00,KY,NaN,NaN,atlanticno5.com
2,2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,NaN,US,NaN,NaN,2015-11-19T22:22:31Z,"Description: Famous Dave's is a Barbecue , Am...",...,NaN,Famous Dave's,NaN,2536040340,98373,USD 10.00 - USD 15.00,WA,5610,NaN,http://famousdaves.com
3,3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,NaN,US,"Coffee & Tea, Restaurants, American, Bar, Pu...",NaN,2016-06-20T17:47:10Z,"The Feve, Oberlin: See 170 unbiased reviews of...",...,/oberlin-oh/mip/the-feve-8940755/menu,The Feve,"mastercard, amex, discover, Yes, visa","[""4407741978"",""4197741978"",""4407753064""]",44074,USD 25.00-40.00,OH,NaN,NaN,http://www.thefeve.com
4,4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,yellowpages.com,US,"Take Out Restaurants, Asian Restaurants, Resta...",NaN,2016-07-23T23:55:05Z,"[""https://www.zagat.com/r/yalla-mediterranean-...",...,/pleasant-hill-ca/mip/yalla-mediterranean-5061...,Quiznos,"master card, visa",9258258644,94523,$,CA,NaN,quiznos,http://www.yallamedi.com/


In [4]:
#use for second datatset

# county_file = "../Resources/CountyLicenseCount.csv"
# county_df = pd.read_csv(county_file)
# county_df.head()

### Transform premise DataFrame

In [63]:
# Create a filtered dataframe from specific columns
vvr_cols = ["index","address", "categories", "city", "country", "name", "phones", "postalCode", "priceRange", "province", "twitter", "websites", "facebookPageURL"]
vvr_transformed= vvr_indexed_df[vvr_cols].copy()

# Rename the column headers
vvr_transformed = vvr_transformed.rename(columns={"postalCode": "postalcode",
                                                  "priceRange": "pricerange",
                                                  "index":"id",
                                                  "facebookPageURL": "facebookpageurl"})

# Clean the data by dropping duplicates and setting the index
# vvr_transformed.drop_duplicates("id", inplace=True)
# vvr_transformed.set_index("id", inplace=True)

vvr_transformed

,id,address,categories,city,country,name,phones,postalcode,pricerange,province,twitter,websites,facebookpageurl
0,0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,US,Village Inn Restaurant and Bakery,7277964988,33759,NaN,FL,NaN,NaN,NaN
1,1,605 W Main St,Caf and Sandwich Place,Louisville,US,Atlantic No. 5,5028833398,40202,USD 25.00-40.00,KY,NaN,atlanticno5.com,NaN
2,2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,Famous Dave's,2536040340,98373,USD 10.00 - USD 15.00,WA,NaN,http://famousdaves.com,NaN
3,3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",44074,USD 25.00-40.00,OH,NaN,http://www.thefeve.com,NaN
4,4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,Quiznos,9258258644,94523,$,CA,quiznos,http://www.yallamedi.com/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18150,18150,879 Smith Ave S,"West St Paul Restaurants, American / Diner / C...",W St Paul,US,The Old Man River Cafe,6.51E+09,55118,NaN,MN,NaN,NaN,NaN
18151,18151,1410 Colonial Life Blvd W,Restaurant,Columbia,US,The Melting Pot,8037318500,29210-7665,NaN,SC,NaN,NaN,NaN
18152,18152,9439 Sheridan St,"American, Smoothies & Juices, Hollywood Restau...",Hollywood,US,Smoothie King,9544313411,33024,NaN,FL,NaN,http://smoothieking.com,NaN
18153,18153,801 N Congress Ave,"[""Restaurants"",""Banquet Halls & Reception Faci...",Boynton Beach,US,Buca Di Beppo,"[""5617344558"",""5617330189""]","[""33426""]",$$,FL,NaN,http://www.bucadibeppo.com,NaN


In [64]:
vvr_transformed = vvr_transformed[~vvr_transformed.name.str.contains("closed", na=False) == True]
vvr_transformed = vvr_transformed[~vvr_transformed.name.str.contains("Closed", na=False) == True]
vvr_transformed


,id,address,categories,city,country,name,phones,postalcode,pricerange,province,twitter,websites,facebookpageurl
0,0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,US,Village Inn Restaurant and Bakery,7277964988,33759,NaN,FL,NaN,NaN,NaN
1,1,605 W Main St,Caf and Sandwich Place,Louisville,US,Atlantic No. 5,5028833398,40202,USD 25.00-40.00,KY,NaN,atlanticno5.com,NaN
2,2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,Famous Dave's,2536040340,98373,USD 10.00 - USD 15.00,WA,NaN,http://famousdaves.com,NaN
3,3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",44074,USD 25.00-40.00,OH,NaN,http://www.thefeve.com,NaN
4,4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,Quiznos,9258258644,94523,$,CA,quiznos,http://www.yallamedi.com/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18150,18150,879 Smith Ave S,"West St Paul Restaurants, American / Diner / C...",W St Paul,US,The Old Man River Cafe,6.51E+09,55118,NaN,MN,NaN,NaN,NaN
18151,18151,1410 Colonial Life Blvd W,Restaurant,Columbia,US,The Melting Pot,8037318500,29210-7665,NaN,SC,NaN,NaN,NaN
18152,18152,9439 Sheridan St,"American, Smoothies & Juices, Hollywood Restau...",Hollywood,US,Smoothie King,9544313411,33024,NaN,FL,NaN,http://smoothieking.com,NaN
18153,18153,801 N Congress Ave,"[""Restaurants"",""Banquet Halls & Reception Faci...",Boynton Beach,US,Buca Di Beppo,"[""5617344558"",""5617330189""]","[""33426""]",$$,FL,NaN,http://www.bucadibeppo.com,NaN


In [68]:
vvr_transformed['postalcode'] = vvr_transformed['postalcode'].str.extract(r'^(\d{5})', expand=False)
vvr_transformed = vvr_transformed[vvr_transformed['postalcode'].notna()]
vvr_transformed

,id,address,categories,city,country,name,phones,postalcode,pricerange,province,twitter,websites,facebookpageurl
0,0,2871 Gulf To Bay Blvd,"American, American (new), Clearwater Restauran...",Clearwater,US,Village Inn Restaurant and Bakery,7277964988,33759,NaN,FL,NaN,NaN,NaN
1,1,605 W Main St,Caf and Sandwich Place,Louisville,US,Atlantic No. 5,5028833398,40202,USD 25.00-40.00,KY,NaN,atlanticno5.com,NaN
2,2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,Famous Dave's,2536040340,98373,USD 10.00 - USD 15.00,WA,NaN,http://famousdaves.com,NaN
3,3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",44074,USD 25.00-40.00,OH,NaN,http://www.thefeve.com,NaN
4,4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,Quiznos,9258258644,94523,$,CA,quiznos,http://www.yallamedi.com/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18149,18149,9 Orchard Hill Sq,Caterers,Staunton,US,Brother's Catering,5402552884,24401,NaN,VA,NaN,http://www.brotherscateringstaunton.com,NaN
18150,18150,879 Smith Ave S,"West St Paul Restaurants, American / Diner / C...",W St Paul,US,The Old Man River Cafe,6.51E+09,55118,NaN,MN,NaN,NaN,NaN
18151,18151,1410 Colonial Life Blvd W,Restaurant,Columbia,US,The Melting Pot,8037318500,29210,NaN,SC,NaN,NaN,NaN
18152,18152,9439 Sheridan St,"American, Smoothies & Juices, Hollywood Restau...",Hollywood,US,Smoothie King,9544313411,33024,NaN,FL,NaN,http://smoothieking.com,NaN


In [70]:
vvr_transformed = vvr_transformed[vvr_transformed['pricerange'].notna()]
# vvr_transformed
vvr_transformed['pricerange']= vvr_transformed['pricerange'].replace('$', '',inplace = True)
vvr_transformed

,id,address,categories,city,country,name,phones,postalcode,pricerange,province,twitter,websites,facebookpageurl
1,1,605 W Main St,Caf and Sandwich Place,Louisville,US,Atlantic No. 5,5028833398,40202,None,KY,NaN,atlanticno5.com,NaN
2,2,3803 9th St S W,"American, Barbecue, Puyallup Restaurants, Barb...",Puyallup,US,Famous Dave's,2536040340,98373,None,WA,NaN,http://famousdaves.com,NaN
3,3,30 S Main St,"Coffee & Tea, Restaurant, Restaurants, Bar, Am...",Oberlin,US,The Feve,"[""4407741978"",""4197741978"",""4407753064""]",44074,None,OH,NaN,http://www.thefeve.com,NaN
4,4,55 Crescent Dr,Sandwich Place and Fast Food Restaurant,Pleasant Hill,US,Quiznos,9258258644,94523,None,CA,quiznos,http://www.yallamedi.com/,NaN
7,7,415 Thompson Ln,"Nashville Restaurants, Glencliff Estates, Rest...",Nashville,US,Lalibela Ethiopian Cuisine,6153320710,37211,None,TN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18101,18101,4 E Madison St,"Restaurants, Baltimore Restaurants, Mount Vern...",Baltimore,US,Feast @ 4 East,4106052020,21202,None,MD,NaN,http://www.4eastmadisoninn.com,NaN
18102,18102,7334 Tylers Corner Dr,"Asian, Sushi, Thai, Restaurant, West Chester R...",W Chester,US,Miso - West Chester,5137791807,45069,None,OH,NaN,http://www.misocincinnati.com/,NaN
18139,18139,8931 N Yates St,"Westminster Restaurants, American, Restaurants...",Westminster,US,Saltgrass Steak House,3034294300,80031,None,CO,saltgrasssteak,NaN,http://facebook.com/308969879719
18143,18143,9729 Belleville Rd,"American, Restaurant, Belleville Restaurants, ...",Belleville,US,O'charley's Belleville Mi - Hours Reviews & More,7346976201,48111,None,MI,NaN,NaN,NaN


### Transform county DataFrame

In [32]:
# county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
# county_transformed = county_df[county_cols].copy()

# # Rename the column headers
# county_transformed = county_transformed.rename(columns={"ID": "id",
#                                                          "County Name (Licensee)": "county_name",
#                                                          "License Count": "license_count",
#                                                          "County ID Code": "county_id"})

# # Set index
# county_transformed.set_index("id", inplace=True)

# county_transformed.head()

### Create database connection

In [7]:
connection_string = "postgres:postgres@localhost:5432/vegan_ram"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# Confirm tables
engine.table_names()

<ipython-input-8-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['vvr']

### Load DataFrames into database

In [20]:
# vvr_transformed.to_sql(name='vvr', con=engine, if_exists='append', index=True)

In [ ]:
# county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)